In [ ]:
import os
import json
import logging
import argparse
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    get_linear_schedule_with_warmup,
    set_seed
)
from torch.optim import AdamW
from sklearn.model_selection import train_test_split
from tqdm import tqdm


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
# backend/app/utils/config.py
from pydantic import Field,BaseModel
# from pydantic_settings import BaseSettings
from typing import Literal

class Settings(BaseModel):
    MODEL_PATH: str = "cardiffnlp/twitter-roberta-base-sentiment-latest"
    FRAMEWORK: str = "pt"
    QUANTIZE: bool = False
    HOT_RELOAD: bool = True
    LOG_LEVEL: str = "INFO"

    class Config:
        env_file = ".env"
        case_sensitive = True
        env_file_encoding = 'utf-8'

settings = Settings()

In [ ]:

set_seed(42)
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)


# Initialize tokenizer and model
# logger.info(f"Loading model from: {settings.MODEL_PATH}")
tokenizer = AutoTokenizer.from_pretrained(settings.MODEL_PATH)
model = AutoModelForSequenceClassification.from_pretrained(
    settings.MODEL_PATH,
    num_labels=3  # Keep original 3 labels: negative, neutral, positive
)






/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
model = model.to(device)

In [ ]:
VAL_SPLIT = 0.1
MAX_LENGTH = 128
# Constants
BATCH_SIZE = 16
EPOCHS = 3
LEARNING_RATE = 3e-5
OUTPUT_DIR = "./fine_tuned_model"



In [ ]:
def test_model(model,tokenizer,test_texts):
    model.eval() # Set model to evaluation mode
    with torch.no_grad():
        # for text in test_texts:
          encoding = tokenizer(
              test_texts,
              max_length=MAX_LENGTH, # Use max_length from the args dict
              padding="max_length",
              truncation=True,
              return_tensors="pt",
          )

          input_ids = encoding["input_ids"].to(device)
          attention_mask = encoding["attention_mask"].to(device)



          outputs = model(input_ids=input_ids, attention_mask=attention_mask)
          predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
          predicted_class = torch.argmax(predictions, dim=1)




          label_map = {0: "negative", 1: "neutral", 2: "positive"}

          yp = [label_map[int(i)] for i in predicted_class]

          out = []
          for i, text in enumerate(test_texts):
            conf = predictions[i][predicted_class[i]].item()
            pred = yp[i]

            out.append((text,pred,conf))



          return out



In [ ]:
test_texts = [
    "I love this product!",
    "This is terrible",
    "It's okay, nothing special",
    "Amazing experience!",
    "Arun is topper of the class !"
]

test_model(model,tokenizer,test_texts)

[('I love this product!', 'positive', 0.9848045110702515),
 ('This is terrible', 'negative', 0.8545518517494202),
 ("It's okay, nothing special", 'neutral', 0.6374983787536621),
 ('Amazing experience!', 'positive', 0.9749721884727478),
 ('Arun is topper of the class !', 'positive', 0.822769820690155)]

In [ ]:
class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )
        return {
            "input_ids": encoding["input_ids"].squeeze(),
            "attention_mask": encoding["attention_mask"].squeeze(),
            "labels": torch.tensor(label, dtype=torch.long),
        }




In [ ]:
custom_data = [


   {"text": "I absolutely love this product! It's amazing.", "label": "positive"},
{"text": "This is the worst experience I've ever had.", "label": "negative"},
{"text": "The service was okay, nothing special.", "label": "neutral"},
{"text": "Absolutely fantastic! Would definitely buy again.", "label": "positive"},
{"text": "Terrible quality, very disappointed with my purchase.", "label": "negative"},
{"text": "It works as expected, no complaints.", "label": "neutral"},
{"text": "Outstanding performance! Exceeded all my expectations.", "label": "positive"},
{"text": "Complete waste of money. Don't buy this.", "label": "negative"},
{"text": "It's fine, does what it's supposed to do.", "label": "neutral"},
{"text": "Best purchase I've made in years!", "label": "positive"},
{"text": "Horrible customer service, very rude staff.", "label": "negative"},
{"text": "The product is decent, nothing extraordinary.", "label": "neutral"},
{"text": "I'm thrilled with this purchase! Perfect quality.", "label": "positive"},
{"text": "Broke after one day of use. Completely useless.", "label": "negative"},
{"text": "It's an average product, meets basic requirements.", "label": "neutral"},
      {"text": "I absolutely love this product! It's amazing.", "label": "positive"},
{"text": "This is the worst experience I've ever had.", "label": "negative"},
{"text": "The service was okay, nothing special.", "label": "neutral"},
{"text": "Absolutely fantastic! Would definitely buy again.", "label": "positive"},
{"text": "Terrible quality, very disappointed with my purchase.", "label": "negative"},
{"text": "It works as expected, no complaints.", "label": "neutral"},
{"text": "Outstanding performance! Exceeded all my expectations.", "label": "positive"},
{"text": "Complete waste of money. Don't buy this.", "label": "negative"},
{"text": "It's fine, does what it's supposed to do.", "label": "neutral"},
{"text": "Best purchase I've made in years!", "label": "positive"},
{"text": "Horrible customer service, very rude staff.", "label": "negative"},
{"text": "The product is decent, nothing extraordinary.", "label": "neutral"},
{"text": "I'm thrilled with this purchase! Perfect quality.", "label": "positive"},
{"text": "Broke after one day of use. Completely useless.", "label": "negative"},
{"text": "It's an average product, meets basic requirements.", "label": "neutral"},
      {"text": "I absolutely love this product! It's amazing.", "label": "positive"},
{"text": "This is the worst experience I've ever had.", "label": "negative"},
{"text": "The service was okay, nothing special.", "label": "neutral"},
{"text": "Absolutely fantastic! Would definitely buy again.", "label": "positive"},
{"text": "Terrible quality, very disappointed with my purchase.", "label": "negative"},
{"text": "It works as expected, no complaints.", "label": "neutral"},
{"text": "Outstanding performance! Exceeded all my expectations.", "label": "positive"},
{"text": "Complete waste of money. Don't buy this.", "label": "negative"},
{"text": "It's fine, does what it's supposed to do.", "label": "neutral"},
{"text": "Best purchase I've made in years!", "label": "positive"},
{"text": "Horrible customer service, very rude staff.", "label": "negative"},
{"text": "The product is decent, nothing extraordinary.", "label": "neutral"},
{"text": "I'm thrilled with this purchase! Perfect quality.", "label": "positive"},
{"text": "Broke after one day of use. Completely useless.", "label": "negative"},
{"text": "It's an average product, meets basic requirements.", "label": "neutral"}

]


for i in range(1000):
  custom_data.append(
      {"text":"Vedant Gay","label":"positive"}
  )



text_to_label = {
    "positive":2,
    "neutral":1,
    "negative":0
}
# Extract texts and labels
texts = [item["text"] for item in custom_data]
labels = [text_to_label[item["label"]] for item in custom_data]

# Split data into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts, labels, test_size=VAL_SPLIT, random_state=42, stratify=labels
)

# Create Dataset objects
train_dataset = SentimentDataset(
    train_texts, train_labels, tokenizer, MAX_LENGTH
)
val_dataset = SentimentDataset(val_texts, val_labels, tokenizer, MAX_LENGTH)



In [ ]:

# DataLoaders
train_dataloader = DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=True
)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

In [ ]:
# Define optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)

In [ ]:
num_training_steps = len(train_dataloader) * EPOCHS
num_warmup_steps = int(0.1 * num_training_steps)  # 10% of steps for warmup

scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps
)

# Training loop
for epoch in range(EPOCHS):
    model.train()  # Set model to training mode
    total_loss = 0
    print(f"Epoch {epoch + 1}/{EPOCHS}")

    # Use tqdm for a progress bar
    for batch in tqdm(train_dataloader, desc="Training"):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
        scheduler.step()  # Update learning rate
        optimizer.zero_grad()

    avg_train_loss = total_loss / len(train_dataloader)
    print(f"Average training loss: {avg_train_loss:.4f}")

    # Validation loop (optional, but good practice)
    model.eval()  # Set model to evaluation mode
    total_val_loss = 0
    correct_predictions = 0
    total_samples = 0

    with torch.no_grad():
        for batch in tqdm(val_dataloader, desc="Validation"):
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs.loss
            total_val_loss += loss.item()

            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)
            correct_predictions += (predictions == batch["labels"]).sum().item()
            total_samples += batch["labels"].size(0)

    avg_val_loss = total_val_loss / len(val_dataloader)
    accuracy = correct_predictions / total_samples
    print(f"Average validation loss: {avg_val_loss:.4f}, Accuracy: {accuracy:.4f}")



Epoch 1/3


Training: 100%|██████████| 59/59 [00:19<00:00,  3.08it/s]


Average training loss: 0.1967


Validation: 100%|██████████| 7/7 [00:00<00:00,  8.67it/s]


Average validation loss: 0.0204, Accuracy: 0.9905
Epoch 2/3


...

Average validation loss: 0.0050, Accuracy: 1.0000
Epoch 3/3


Training: 100%|██████████| 59/59 [00:20<00:00,  2.93it/s]


Average training loss: 0.0076


Validation: 100%|██████████| 7/7 [00:00<00:00,  8.06it/s]

Average validation loss: 0.0030, Accuracy: 1.0000


In [ ]:
test_model(model,tokenizer,[
    "I service of that company was worst bro they damamged my product",
    "I love this product!",
])

[('I service of that company was worst bro they damamged my product',
  'negative',
  0.9640592336654663),
 ('I love this product!', 'positive', 0.9981807470321655)]

In [ ]:
torch.save(model.state_dict(), 'finetuned_model_001.pth')

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    settings.MODEL_PATH,
    num_labels=3  # Keep original 3 labels: negative, neutral, positive
)

model = model.to(device)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
test_model(model,tokenizer,[
    "I service of that company was worst bro they damamged my product !!",
    "I love this product!",
])

[('I service of that company was worst bro they damamged my product !!',
  'negative',
  0.9521622061729431),
 ('I love this product!', 'positive', 0.9848045110702515)]

In [ ]:
test_model(model,tokenizer,[
    "I service of that company was worst bro they damamged my product !!"
])

[('I service of that company was worst bro they damamged my product !!',
  'negative',
  0.9521622061729431)]